In [1]:
source('https://raw.githubusercontent.com/sametsoekel/nonsystematic_workspace/master/space/nn_pipeline_0812.r')

In [3]:
data_sl <- data.table()

for(i in c(LETTERS[1:15],letters[1:22])){
    charmi <- sample(c(TRUE,FALSE),size = 1)
    ortsec <- sample(200:500,size = 1)
    sdsec <- sample(100:1000,size = 1)
    if(charmi){
        data_sl[,(i) := sample(c(LETTERS,NA),size = 2e6,replace = T)]
    }else{
        data_sl[,(i) := sample(c(rnorm(n = 2e4,mean = ortsec,sd = sdsec),NA),size = 2e6,replace = T)]
    }
}

data_sl[,default := sample(c('No','Yes'),size = 2e6,prob = c(.93,.07),replace = T)]

In [4]:
set.seed(12)
traindex <- sample(1:nrow(data_sl),nrow(data_sl)*0.66)
train_set <- data_sl[traindex,]
test_set <- data_sl[-traindex,] %>% select(-default) 

In [5]:
train_ok <- train_set %>% make_frame(label = 'default')
test_ok <- test_set %>% make_frame(label = 'default',test = T,train_frame = train_ok)

Data sparsed.

Scaling factors calculated.

Missing values labeled.

Data scaled.



ERROR: Error in distincts[[i]]: seviye 1'de böyle bir indis yok



In [6]:
sprs <- sparse_data(data_sl)

In [7]:
scl <- get_scaling_factors(sprs)

In [9]:
scld <- scale_data(sprs,scaling_factors = scl)

In [14]:
get_distincts <- function(scaled_data){
    df <- copy(scaled_data[['data']])
    
    catcols <- scaled_data[['cat_cols']]
    
    distincts <- list()
    for(i in catcols){
        distinct_values <- df[,unique(.SD),.SDcols = i] %>% pull %>% as.character
        distincts[[i]] <- c(distinct_values,'Missing')
        df[,(i) := lapply(.SD,function(x) as.character(x)),.SDcols = i]
        df[,(i) := lapply(.SD,function(x) ifelse(is.na(x),'Missing',x)),.SDcols = i]
        df[,(i) := lapply(.SD,function(x) factor(x,levels = distincts[[i]])),.SDcols = i]
    }
    
    

    scaled_data[['cat_distincts']] <- distincts
    
    scaled_data[['data']] <- df
                          
    scaled_data
}

In [28]:
df <- copy(scld[['data']])
    
catcols <- scld[['cat_cols']]

distincts <- list()
for(i in catcols){
    distinct_values <- df[,unique(.SD),.SDcols = i] %>% pull %>% as.character
    distincts[[i]] <- c(distinct_values,'Missing')
    df[,(i) := lapply(.SD,function(x) as.character(x)),.SDcols = i]
    df[,(i) := lapply(.SD,function(x) ifelse(is.na(x),'Missing',x)),.SDcols = i]
    df[,(i) := lapply(.SD,function(x) factor(x,levels = c(distinct_values,'Missing'))),.SDcols = i]
}



#scaled_data[['cat_distincts']] <- distincts

#scaled_data[['data']] <- df

#scaled_data

In [30]:
1.5e6

[1] 1500000

In [26]:
distinct_values

[1] "I"       "A"       "R"       "F"       "H"       "T"       "J"      
 [8] "G"       "Missing" "N"       "B"       "P"       "L"       "D"      
[15] "X"       "K"       "V"       "U"       "C"       "S"       "O"      
[22] "Y"       "M"       "Z"       "Q"       "W"       "E"

In [27]:
distincts[[i]]

[1] "I"       "A"       "R"       "F"       "H"       "T"       "J"      
 [8] "G"       NA        "N"       "B"       "P"       "L"       "D"      
[15] "X"       "K"       "V"       "U"       "C"       "S"       "O"      
[22] "Y"       "M"       "Z"       "Q"       "W"       "E"       "Missing"